In [24]:
import pickle
import time

with open("../../data/article_texts.txt",'rb') as f:
    texts = pickle.load(f, encoding="UTF-8")
with open("../../data/english_anecs_list.pickle", "rb") as f:
    english_anecs_list = pickle.load(f, encoding="UTF-8")
with open("../../data/ids_to_labels.pickle", "rb") as f:
    ids_to_labels = pickle.load(f, encoding="utf-8")
with open("../../data/labels_to_ids.pickle", "rb") as f:
    labels_to_ids = pickle.load(f, encoding="utf-8")
with open("../../data/unique_labels.pickle", "rb") as f:
    unique_labels = pickle.load(f, encoding="UTF-8")
with open("../../data/translated_anecs.txt", "r") as f:
    translated_anecs = f.read().replace("<unk> ", "").replace("♪ ", "").split("\n")
with open("../../data/translated_anecs_prepared.pickle", "rb") as f:
    translated_anecs_prepared = pickle.load(f)
with open("../../data/english_anecs_prepared.pickle", "rb") as f:
    english_anecs_prepared = pickle.load(f)
with open("../../data/glove_vectors.pickle", "rb") as f:
    glove_vectors = pickle.load(f)

In [25]:
print(translated_anecs[0])

Tell me, is that your Jaguar parked outside? Yes I 'll finish. 


In [26]:
text = ['SpaceX Starship Blows Up Minutes After Launch',
 'SpaceX’s Starship rocket, the most powerful ever built, blasted off on an unpiloted maiden flight Thursday, flying for more than two minutes before exploding. What do you think?']

In [27]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('dslim/bert-base-NER')

def tokenize(data:str):
    inputs = tokenizer(data, return_tensors="pt", truncation=True, padding=True)
    return inputs
def ids_to_tokens(text_input):
    return tokenizer.convert_ids_to_tokens(text_input)

In [28]:
from transformers import BertForTokenClassification
import torch

class BertModel(torch.nn.Module):
    def __init__(self):
        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('dslim/bert-base-NER', num_labels=len(unique_labels),
                                                               ignore_mismatched_sizes=True)

    def forward(self, input_ids, label=None):
        output = self.bert(labels=label, input_ids=input_ids, return_dict=False)
        return output



model = BertModel()

model.load_state_dict(torch.load('../models/bert_trainedNEREnglish', map_location=torch.device('cpu')))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([17, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([17]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [29]:
import numpy as np
def get_ners(text:str) -> list:
    output = []
    B = np.asarray([tokenizer(text.replace("-", ""))["input_ids"]]).reshape(1,1,-1)
    logits = model(torch.as_tensor(np.array(B))[0])[0]
    for j in range(logits.shape[0]):
        #print(logits[i])
        logits_clean = logits[j].argmax(dim=1)
        words = text.replace("-", "").split()
        tokenized_sentence = ids_to_tokens(tokenizer(text.replace("-", ""))["input_ids"])
        for i in range(len(logits_clean[1:-1])):
            print(tokenized_sentence[i+1], ids_to_labels[logits_clean[i+1].item()])
        #print([ids_to_labels[x.item()] for x in logits_clean])
        i = 1
        for elem in logits_clean[1:-1]:
            if i > 1 and (tokenized_sentence[i][:2] == "##" or ids_to_labels[elem.item()][0] == "I") :
                if tokenized_sentence[i][:2] == "##":
                    output[-1]["word"] += tokenized_sentence[i][2:]
                else:
                    output[-1]["word"] += tokenized_sentence[i]
            else:
                output.append({"word":tokenized_sentence[i], "entity":ids_to_labels[elem.item()]})
            i += 1
        """
        for el in logits_clean:
            if i == len(words):
                break
            elem = logits_clean[k]
            if k+1 <= len(logits_clean) and tokenized_sentence[k][:2] == "##":
                if elem.item() == "O":
                    label = logits_clean[k]
                else:
                    label = elem.item()
                output.append({"word":words[i], "entity":ids_to_labels[label]})
                k+=2
            else:
                output.append({"word":words[i], "entity":ids_to_labels[elem.item()]})
                k+=1
            i += 1
        """
    return output
out = get_ners(texts[0][1])
print(out)
temp = ""
for elem in out:
    temp += elem["word"] + " "
print(temp)

Space B-org
##X I-org
’ O
s O
Stars O
##hip O
rocket O
, O
the O
most O
powerful O
ever O
built O
, O
blasted O
off O
on O
an O
un O
##pi O
##lot O
##ed O
maiden O
flight O
Thursday B-tim
, O
flying O
for O
more O
than O
two O
minutes O
before O
exploding O
. O
What O
do O
you O
think O
? O
[{'word': 'SpaceX', 'entity': 'B-org'}, {'word': '’', 'entity': 'O'}, {'word': 's', 'entity': 'O'}, {'word': 'Starship', 'entity': 'O'}, {'word': 'rocket', 'entity': 'O'}, {'word': ',', 'entity': 'O'}, {'word': 'the', 'entity': 'O'}, {'word': 'most', 'entity': 'O'}, {'word': 'powerful', 'entity': 'O'}, {'word': 'ever', 'entity': 'O'}, {'word': 'built', 'entity': 'O'}, {'word': ',', 'entity': 'O'}, {'word': 'blasted', 'entity': 'O'}, {'word': 'off', 'entity': 'O'}, {'word': 'on', 'entity': 'O'}, {'word': 'an', 'entity': 'O'}, {'word': 'unpiloted', 'entity': 'O'}, {'word': 'maiden', 'entity': 'O'}, {'word': 'flight', 'entity': 'O'}, {'word': 'Thursday', 'entity': 'B-tim'}, {'word': ',', 'entity': 'O'}

In [30]:
def get_embeddings(list_of_tags: list):
    emeddings = []
    for tag in list_of_tags:
        try:
            #print(tokenizer.convert_ids_to_tokens(tokenizer(tag["word"])["input_ids"][1]))
            embed = glove_vectors[tag["word"]]
            emeddings.append({'entity': tag["entity"] , 'word': tag["word"], "embedding": embed})
        except:
            emeddings.append({'entity': tag["entity"] , 'word': tag["word"], "embedding": glove_vectors["base"]})
            #print("Broken embedding", tag["word"], tag["entity"])
    return emeddings
def get_non_o(ner_words):
    a = []
    for x in ner_words:
        if x["entity"] != "O" :
            a.append(x)
    return a


In [31]:
from numpy.linalg import norm
from collections import Counter

def count_cos_embeddings(text_embeddings, anec_embeddings) -> (float, dict):
    suitable_pairs = []
    anec_unique_tags_counter = Counter()
    embedding_cosine_sum = 0
    for embedding in text_embeddings:
        cosines = []
        pair = ""
        simmilarity_tags = {}
        for embed in anec_embeddings:
            anec_unique_tags_counter[embed["entity"]] += 1
            #print(f"{embed['word']}:{embed['entity']}", f"{embedding['word']}:{embedding['entity']}")
            if embed["entity"] == embedding["entity"]:
                v1 = embedding["embedding"]
                v2 = embed["embedding"]
                cos = np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
                cosines.append((cos, embed["word"], embed["entity"] ))
        simmilarity_tags[embedding["word"]] = cosines
        suitable_pairs.append(simmilarity_tags)
    top_similarity = {}
    for word_dict in suitable_pairs:
        word = list(word_dict.keys())[0]
        top = (-10, "")
        key = None
        for sim in word_dict[word]:
            if top[0] < sim[0]:
                top = (sim[0], word)
                key = sim[1]
        if key:
            top_similarity[key] = top
    #print(top_similarity)
    #print(suitable_pairs)
    for key in top_similarity.keys():
        embedding_cosine_sum += top_similarity[key][0]
    embedding_cosine_sum /= len(top_similarity.keys()) if len(top_similarity.keys())>5 else 5
    return embedding_cosine_sum, top_similarity

In [9]:
filtred_anec = get_non_o(get_ners(english_anecs_list[30]))
filtred_text = get_non_o(get_ners(text))
anec_embeddings = get_embeddings(filtred_anec)
text_embeddings = get_embeddings(filtred_text)
new_cosine, new_simmilarity = count_cos_embeddings(text_embeddings, anec_embeddings)
#print(filtred_anec, filtred_text)
print(new_simmilarity)
#print(text_embeddings, anec_embeddings)

Dr B-per
. I-per
Cut I-per
##ter I-per
is O
the O
local O
V O
##eter O
##ina O
##rian O
, O
known O
for O
his O
w O
##ry O
humor O
. O
He O
surpassed O
himself O
one O
summer O
day O
when O
a O
city O
dog O
was O
brought O
to O
him O
after O
an O
encounter O
with O
a O
p O
##or O
##cup O
##ine O
. O


AttributeError: 'list' object has no attribute 'replace'

In [32]:
anec = english_anecs_list[5]
text = texts[12][1]
#get_ners(anec)
get_non_o(get_ners("Lawyers for FTX founder Sam Bankman-Fried on Monday filed motions to dismiss the US government’s fraud charges against him"))

Law O
##yers O
for O
F B-org
##T B-org
##X I-org
founder O
Sam B-per
Bank I-per
##man I-per
##F I-per
##ried I-per
on O
Monday O
filed O
motions O
to O
dismiss O
the O
US B-geo
government O
’ O
s O
fraud O
charges O
against O
him O


[{'word': 'FTX', 'entity': 'B-org'},
 {'word': 'SamBankmanFried', 'entity': 'B-per'},
 {'word': 'US', 'entity': 'B-geo'}]

In [34]:
english_anecs_prepared

[]

In [12]:
from tqdm.notebook import tqdm
import random
best_anec = None
best_cos = -10
best_text_ner = None
best_simmilarity = None
best_filtered = None
swear_flag = True

if swear_flag:
    resulted_anec_list = english_anecs_prepared + translated_anecs_prepared
else:
    resulted_anec_list = english_anecs_prepared

filtred_text = get_non_o(get_ners(text))
text_embeddings = get_embeddings(filtred_text)
for anec_embed, anec in tqdm(resulted_anec_list):
    try:
        new_cosine, new_simmilarity = count_cos_embeddings(text_embeddings, anec_embed)
        if new_cosine > best_cos:
            best_cos = new_cosine
            best_filtered = anec_embed
            best_text_ner = filtred_text
            best_anec = anec
            best_simmilarity = new_simmilarity
    except Exception as e:
        print(e)
print(best_anec,best_cos, best_text_ner, best_filtered, best_simmilarity)

The O
Tibetan B-gpe
spiritual O
leader O
the O
Dal B-per
##ai B-per
Lama I-per
apologized O
Monday O
after O
a O
video O
that O
showed O
him O
asking O
a O
boy O
to O
suck O
his O
tongue O
triggered O
a O
back O
##lash O
on O
social O
media O
. O
What O
do O
you O
think O
? O


  0%|          | 0/4158 [00:00<?, ?it/s]

Some of the most tactful people on Earth are English.  One  office supervisor called a secretary in to give her the bad news  that she was being fired.  He started the conversation with:  "Miss Symthe, I really don't know how we're going to get along  without you, but starting Monday, we're going to try. 0.4 [{'word': 'Tibetan', 'entity': 'B-gpe'}, {'word': 'DalaiLama', 'entity': 'B-per'}] [{'entity': 'B-gpe', 'word': 'English.', 'embedding': array([-0.85078  ,  0.14052  ,  0.19218  , -0.56941  ,  0.012045 ,
       -0.13574  ,  0.026476 ,  0.63774  ,  0.13145  , -1.5406   ,
        0.022069 , -0.093937 , -0.0075906,  0.18398  ,  0.098117 ,
        0.34659  , -0.29379  , -0.023845 ,  0.34028  , -0.16479  ,
       -0.4635   , -0.7426   , -0.098289 , -0.66905  , -0.20917  ,
       -0.097382 , -0.028347 ,  0.5289   , -0.1507   ,  0.52132  ,
        0.096284 , -0.20201  , -0.23403  ,  0.35867  ,  0.14002  ,
        0.13554  ,  0.15319  ,  0.32805  ,  0.020936 ,  0.1657   ,
       -0.0089688

In [13]:
words = best_anec.split()
key_words = best_simmilarity.keys()
print(key_words)
for i in range(len(words)):
    if words[i] in key_words:
        words[i] = best_simmilarity[words[i]][1]
resulted_text = " ".join(words)
print(resulted_text)

dict_keys(['English.', "don't"])
Some of the most tactful people on Earth are Tibetan One office supervisor called a secretary in to give her the bad news that she was being fired. He started the conversation with: "Miss Symthe, I really DalaiLama know how we're going to get along without you, but starting Monday, we're going to try.
